## Import Statements
Import all the packages needed for the script

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet import ResNet50
from tensorflow.python.keras.layers import Flatten, Dense
from tensorflow.python.keras.models import Sequential

## Prepare and process the training and validation dataset

In [ ]:
training_ds = tf.keras.utils.image_dataset_from_directory(
    directory='dataset',
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    label_mode='categorical',
    image_size = (224, 224),
    batch_size = 32
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory='dataset', 
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    label_mode='categorical',
    image_size = (224, 224),
    batch_size = 32
)

class_names = training_ds.class_names
print(class_names)

## Implement the Convolutional Neural Network 

This is a sequential neural network that contains a ResNet50 layer, a Flattening layer and 2 Dense layers

In [ ]:
resnet_model = Sequential()

# create resnet model
pretrained_model = ResNet50(include_top=False, weights='imagenet', pooling='max', input_shape=(224, 224, 3))

# dont train the resnet again
for layer in pretrained_model.layers:
    layer.trainable = False

# adding layers to the neural network
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dense(2, activation='sigmoid'))

## Compile your model

In [ ]:
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

## Train(fit) the model with your dataset


In [ ]:
resnet_model.fit(training_ds, validation_data=validation_ds, epochs=6, batch_size=32)

## Evaluate the model to get the true metrics using a test dataset

In [ ]:
# test_loss, test_accuracy, test_precision, test_recall, test_f1_score = resnet_model.evaluate(test_images, test_labels)

# print('Test accuracy:', test_accuracy)
# print('Test precision:', test_precision)
# print('Test recall:', test_recall)
# print('Test f1-score:', test_f1_score)

## Use the model to predict an image
Firstly load and process the image and then use it to predict

In [ ]:
import numpy as np
from PIL import Image

# processing the image
img = Image.open('test_image.jpg')
img = img.resize((224, 224))
img = np.array(img) # convert image to array 
img = img / 255.0  # normalize pixel values between 0 and 1
img = np.expand_dims(img, axis=0)  # add batch dimension

# Make a prediction
predictions = resnet_model.predict(img)
# print(predictions)
predicted_class = np.argmax(predictions)
# print(predicted_class)

# Print the predicted class index
print('This image is: ' + class_names[predicted_class])

## Save the model

In [ ]:
resnet_model.save('my_resnet_model')